###  Chain of Thought (CoT) Prompting
- CoT guides the model to break down problems into smaller, manageable, and logical steps
- Use CoT for tasks that are logic-heavy but do not require up-to-date or external information.
-  Focuses on decomposing problems into intermediate, logical,, human-interpretable reasoning steps.

User query:

“I’m traveling from Bangalore to Delhi for 3 days. Find the cheapest flight, suggest a hotel within ₹4,000/night, and estimate total trip cost.”

In [27]:
from langchain_core.tools import tool 


@tool 
def search_flights(origin:str , destination:str , date:str) -> dict:
    """Search for the cheapest flights"""
    return {'airline':'Indigo' , 'price':'4500' , 'arrival':'4.00' , 'departure':'6.00'}

@tool
def search_hotels(city:str , max_price: int)-> dict:
    """Search for the best hotel under budget"""
    return {'hotel':'Novotel' , "price_per_night": 3500,"rating": 4.2}

@tool
def  calculate_total_cost(flight_price :int , hotel_price: int , night: int )-> int:
    """Returns the totall cost"""
    return flight_price +  (hotel_price * night)

@tool
def format_flight_summary(hotel :dict , flight : dict ,  total_cost: int)-> str:
    """Formats flight plan summary"""
    return (
        f"FLIGHT: {flight['airline']}, {flight['price']},{flight['departure']}, {flight['arrival']}\n"
        f"HOTEL :{hotel['hotel']}, {hotel['price_per_night']}/night , {hotel['rating']}\n"
        f"TOTAL COST: {total_cost}"
    )

In [28]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os 
from dotenv import load_dotenv
load_dotenv()

llm = ChatGroq(
    model="llama-3.1-8b-instant",   
    api_key=os.getenv("GROQ_API_KEY"),
    temperature=0
)

llm_with_tools = llm.bind_tools(
    [search_flights , search_hotels , calculate_total_cost , format_flight_summary] 
)

In [29]:
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are an intelligent travel planning assistant.\n\n"

     "Your responsibilities:\n"
     "1. Carefully understand the user's travel intent, cities, dates, duration, and budget.\n"
     "2. Break the task into logical steps and reason through each step internally.\n"
     "3. Use available tools whenever external data, calculations, or structured results are required.\n"
     "4. Never assume prices, availability, or schedules — always call the appropriate tools.\n"
     "5. Validate all inputs and outputs for consistency and correctness.\n"
     "6. If required information is missing, ask a clear follow-up question before proceeding.\n"
     "7. Combine tool outputs into a coherent, user-friendly final response.\n\n"

     "Tool usage rules:\n"
     "- Call tools for flight search, hotel search, and cost calculation.\n"
     "- Do not hallucinate or fabricate data.\n"
     "- If a tool fails or returns empty results, gracefully inform the user and suggest alternatives.\n\n"

     "Response formatting guidelines:\n"
     "- Provide a structured itinerary with clearly labeled sections.\n"
     "- Include prices, timings, and key details.\n"
     "- End with a concise recommendation or summary.\n\n"

     "Behavioral constraints:\n"
     "- Be polite, professional, and concise.\n"
     "- Avoid unnecessary verbosity.\n"
     "- Do not reveal internal reasoning steps.\n"
     "- Once all required information is gathered and the final answer is generated, stop responding.\n"
     "- If you are uncertain, clearly state that and ask for clarification.\n"
    ),
    ("user", "{question}")
])


### response.tool_calls :list of tool invocation requests generated by the LLM when it decides it needs  functions/tools to complete the task.

-  Each item represents one tool call with:
- Which tool to call
- What arguments to pass
- A unique call ID

In [30]:
def run_agent(question: str):
    messages = prompt.format_messages(question=question)
    
    while True:
        response = llm_with_tools.invoke(messages)
        
        # Add assistant message
        messages.append(response)

        if response.tool_calls:
            for tool_call in response.tool_calls:
                tool_name = tool_call["name"]
                args = tool_call["args"]
                
                tool_fn = {
                    "search_flights": search_flights,
                    "search_hotels": search_hotels,
                    "calculate_total_cost": calculate_total_cost,
                    "format_flight_summary": format_flight_summary
                }[tool_name]
                
                result = tool_fn.invoke(args)
                
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call["id"],
                    "content": str(result)
                })
        else:
            
            return response.content


In [ ]:
result = run_agent("I’m traveling from Bangalore to Delhi for 3 days. Find the cheapest flight, suggest a hotel within ₹4,000/night, and estimate total trip cost.")
print(result)


Here's a structured itinerary for your 3-day trip from Bangalore to Delhi:

**Flight Details:**

* Airline: Indigo
* Price: ₹ 4500
* Departure: 6:00 AM
* Arrival: 4:00 PM

**Hotel Details:**

* Hotel: Novotel
* Price per night: ₹ 3500
* Rating: 4.2/5

**Total Trip Cost:**

* Estimated total cost for the 3-day trip: ₹ 11,000

This includes the cost of the flight and the hotel stay for 3 nights. Please note that these prices are estimates and may vary based on various factors such as availability, season, and other travel requirements.
